In [1]:
from notebooks.__init__ import *
%load_ext autoreload
%autoreload 2

# Load a config

In [2]:
# method_overrides = ["+method/end_to_end=disk"]
method_overrides = ["+method/detector=boness_st/default", 
                    "+method/descriptor=disk"]
# method_overrides = ["+method/detector=superpoint",
#                     "+method/descriptor=disk"]

In [3]:
with initialize_config_dir(config_dir='/home/konstantin/personal/Summertime/config', version_base='1.1'):
    cfg = compose(overrides=method_overrides + [
        "+test/dataset=hpatches",
        "+test/evaluation=classical_metrics",
        "+test/experiment=default",
        "+mock=True"
    ])

    OmegaConf.set_struct(cfg, False)

    cfg_wrapper = HydraConfigWrapper(cfg)

    cfg_wrapper.print()
    



Configuration file:
------------------------------------------------------------------------------------------
method:
  detector:
    name: boness_st
    model:
      input_channels: 3
      down:
      - 16
      - 32
      - 64
      - 64
      - 64
      up:
      - 64
      - 64
      - 64
      - 1
    base_detector:
      name: shi_tomasi
      sobel_size: 3
      window_size: 3
      window_cov: 2
      nms_size: 5
      localize: true
      score_thresh: 0.0
  descriptor:
    name: disk
    checkpoint_url: file:///home/konstantin/personal/Summertime/source/baselines/disk/disk/depth-save.pth
test:
  dataset:
    name: HPatches
    root_path: /mnt/sda/datasets/HPatches
    csv_rel_path: test.csv
    pairs_csv_rel_path: test_pairs.csv
    loader:
      num_workers: 2
      num_samples: 8
    pairs_loader:
      batch_size: 2
      num_workers: 8
      num_samples: 4
  evaluation:
    task: classical_metrics
    'n': 2048
    feature_dir_path: /home/konstantin/personal/features


# Instantiate a test task

In [5]:
from source.test_task import TestTask

feature_dir_path = Path("/home/konstantin/personal/features/notebooks")
test_task = TestTask.from_config(cfg, None, feature_dir_path)

if cfg.test.experiment.accelerator == 'gpu':
    test_task.to(torch.device('cuda'))

Loaded checkpoint for disk


# Instantiate a dataset and get loaders

In [6]:
data_module = DataModule(test_dataset_cfg=cfg.test.dataset, 
                         evaluation_cfg=cfg.test.evaluation,
                         feature_dir_path=test_task.feature_dir_path)
data_module.setup(stage='test')

In [7]:
image_loader, feature_loader = data_module.test_dataloader()

# Compute features and calculate metrics

In [8]:
test_task.on_test_epoch_start()

In [9]:
with torch.no_grad():
    if test_task.detector is not None:
        test_task.detector.eval()

    if test_task.descriptor is not None:
        test_task.descriptor.eval()
        
    if test_task.end_to_end is not None:
        test_task.end_to_end.eval()

    for i, batch in enumerate(image_loader):
        if cfg.test.experiment.accelerator == 'gpu':
            for k in batch.keys():
                if isinstance(batch[k], torch.Tensor):
                    batch[k] = batch[k].to(torch.device('cuda'))
        
        test_task.test_step(batch, i, 0)

/opt/conda/lib/python3.11/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/opt/conda/lib/python3.11/site-packages/torch/nn/functional.py:4373: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


In [ ]:
results_list = []

for i, batch in enumerate(feature_loader):
    test_task.test_step(batch, i, 1)

results = np.concatenate(test_task.test_step_outputs, axis=0)

In [14]:
rep = results[:, 3:13].astype(np.float32).mean(axis=0)
mma = results[:, 13:].astype(np.float32).mean(axis=0)

print("Repeatability: ", rep)
print("Mean matching accuracy: ", mma)

Repeatability:  [0.10519426 0.21751387 0.30479237 0.3694466  0.4278297  0.4663459
 0.4907268  0.5051263  0.5143215  0.52120703]
Mean matching accuracy:  [0.30874723 0.5044017  0.56879395 0.59358555 0.61104554 0.6312906
 0.65241814 0.6670246  0.6764157  0.6884844 ]


In [15]:
import shutil

shutil.rmtree(test_task.feature_dir_path)